# Fake Portfolio Data

I need to build some fake data for X days that for each day includes:
- Factor Returns 
- Weights

In [364]:
import polars as pl
import numpy as np
from datetime import date, timedelta, datetime
np.random.seed(19981)

In [377]:
size = 1_0000
days = 30
current_date = date.today()
main_df = None
main_factors = None
scale = 100

for i in range(30):
    # Random Normal
    styles = list(sorted(['MOMENTUM', 'VOLATILITY', 'GROWTH', 'YIELD']))
#     styles = list(sorted(['MOMENTUM']))
    data = { x: np.random.normal(0,1,size) for x in styles }

    # Ramdom Categorical
    regions = list( sorted(['US', 'EU', 'ASIA', 'LATAM', 'AU']))
    regions = list( sorted(['US', 'EU', 'ASIA']))
#     regions = []
    sectors = list(sorted(['TECH', 'FINANCE', 'RETAIL', 'BLUE', 'GREEN']))
#     sectors = []
    
    data['R'] = np.random.choice(regions, size)
    data['S'] = np.random.choice(sectors, size)

    factor_names = data.keys()
#     data['ALPHA'] = np.zeros(size) # np.random.normal(0,1, size)
    data['ALPHA'] = np.random.normal(0,.5, size)
    data['DATE'] = [current_date] * size

    df = pl.DataFrame(data)
    df = pl.concat([df[['DATE', 'ALPHA']],df[['R', 'S']].to_dummies(), df.drop(['DATE', 'ALPHA','R', 'S'])], how='horizontal')    
#     df = pl.concat([df[['DATE', 'ALPHA']], df.drop(['DATE', 'ALPHA','R']), df[['R']].to_dummies()], how='horizontal')    
#     df = pl.concat([df[['DATE', 'ALPHA']], df.drop(['DATE', 'ALPHA'])], how='horizontal')    

    names = regions+sectors+styles
    factors = np.random.normal(0,scale, len(names))
    factors = np.round(factors/factors.sum())
    factors = np.insert(factors,0,1)
    factors = factors.reshape(len(factors),1)
    
    names.insert(0,'ALPHA')
    
    factors_df = pl.DataFrame(factors, schema=names, orient="col")
    ret = np.dot(factors_df, df.drop(['DATE']).to_numpy().T) #/ ((len(names)-1)+scale*(len(names)-1))
    
    factors_df = factors_df.with_columns(pl.lit(current_date).dt.date().alias('DATE'))
    factors_df = pl.concat([factors_df[['DATE']], factors_df.drop(['DATE'])],  how='horizontal')
    df = df.hstack([pl.Series('RETURN', ret.reshape(size))])
    
    if main_df is None:
        main_df = df
        main_factors = factors_df
    else:
        main_df.extend(df)
        main_factors.extend(factors_df)
        
    current_date += timedelta(days=1)

In [385]:
# zzz = np.random.normal(0,scale, len(names))
# zzz.sum()
# zzz / zzz.sum()

array([-14.25448361,   2.10228743,   5.43789533, -16.49017609,
       -10.6553208 ,  10.84876855,  -0.29611523,   6.54510623,
        10.28496622,   7.67100827,  -8.78015883,   6.96881048,
         1.61741204])

In [378]:
main_factors.head()

DATE,ALPHA,ASIA,EU,US,BLUE,FINANCE,GREEN,RETAIL,TECH,GROWTH,MOMENTUM,VOLATILITY,YIELD
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-05-21,1.0,10.0,-1.0,1.0,-5.0,1.0,3.0,-4.0,-1.0,1.0,-7.0,-1.0,4.0
2023-05-22,1.0,-1.0,-0.0,1.0,-1.0,-0.0,0.0,-0.0,2.0,0.0,-1.0,0.0,1.0
2023-05-23,1.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2023-05-24,1.0,0.0,-0.0,-0.0,-0.0,0.0,1.0,0.0,0.0,-1.0,1.0,-0.0,0.0
2023-05-25,1.0,-1.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-0.0,-0.0,2.0,1.0,-0.0


In [379]:
main_df.head()

DATE,ALPHA,R_ASIA,R_EU,R_US,S_BLUE,S_FINANCE,S_GREEN,S_RETAIL,S_TECH,GROWTH,MOMENTUM,VOLATILITY,YIELD,RETURN
date,f64,u8,u8,u8,u8,u8,u8,u8,u8,f64,f64,f64,f64,f64
2023-05-21,0.120394,1,0,0,0,1,0,0,0,-0.118844,-0.442826,0.048512,-0.509431,12.015098
2023-05-21,0.466741,0,1,0,0,0,1,0,0,-0.35689,-0.387665,-1.160233,-0.285834,4.8404
2023-05-21,-0.811256,0,1,0,0,0,0,1,0,-0.066273,0.190383,-0.427118,1.877026,0.725009
2023-05-21,0.070642,1,0,0,0,0,0,0,1,0.138358,-0.945388,-0.846304,-0.63404,14.13686
2023-05-21,0.398148,0,0,1,1,0,0,0,0,-1.933958,-1.972134,0.649253,-0.839043,4.2637


In [380]:
main_factors.write_csv('main_factors.csv')
main_df.write_csv('main_df.csv')


print(1)

1
